<a href="https://colab.research.google.com/github/alexandreib/QuantDesign/blob/main/QD%20%7C%20SP500%20Portfolio%20Allocation.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

#**Sharpe Ratio Optimization on returns, on S&P 500 Past Returns**



## Imports

In [97]:
import pandas as pd
import numpy as np
import yfinance as yf
import seaborn as sns
import scipy as sp
import cvxpy as cp

# Creating the DataFrame

## Scapping SP500 Constituents from Wikipedia

In [98]:
# Download the S&P 500 constituents from Wikipedia
try:
    table=pd.read_html('https://en.wikipedia.org/wiki/List_of_S%26P_500_companies')
    df = table[0]
    tickers = df['Symbol'].tolist()
except Exception as e:
    print(f"Error downloading S&P 500 tickers: {e}")
    tickers = [] # Handle the error gracefully, e.g., provide a default list

# Print or use the tickers list
print(tickers)

['MMM', 'AOS', 'ABT', 'ABBV', 'ACN', 'ADBE', 'AMD', 'AES', 'AFL', 'A', 'APD', 'ABNB', 'AKAM', 'ALB', 'ARE', 'ALGN', 'ALLE', 'LNT', 'ALL', 'GOOGL', 'GOOG', 'MO', 'AMZN', 'AMCR', 'AMTM', 'AEE', 'AEP', 'AXP', 'AIG', 'AMT', 'AWK', 'AMP', 'AME', 'AMGN', 'APH', 'ADI', 'ANSS', 'AON', 'APA', 'AAPL', 'AMAT', 'APTV', 'ACGL', 'ADM', 'ANET', 'AJG', 'AIZ', 'T', 'ATO', 'ADSK', 'ADP', 'AZO', 'AVB', 'AVY', 'AXON', 'BKR', 'BALL', 'BAC', 'BAX', 'BDX', 'BRK.B', 'BBY', 'TECH', 'BIIB', 'BLK', 'BX', 'BK', 'BA', 'BKNG', 'BWA', 'BSX', 'BMY', 'AVGO', 'BR', 'BRO', 'BF.B', 'BLDR', 'BG', 'BXP', 'CHRW', 'CDNS', 'CZR', 'CPT', 'CPB', 'COF', 'CAH', 'KMX', 'CCL', 'CARR', 'CTLT', 'CAT', 'CBOE', 'CBRE', 'CDW', 'CE', 'COR', 'CNC', 'CNP', 'CF', 'CRL', 'SCHW', 'CHTR', 'CVX', 'CMG', 'CB', 'CHD', 'CI', 'CINF', 'CTAS', 'CSCO', 'C', 'CFG', 'CLX', 'CME', 'CMS', 'KO', 'CTSH', 'CL', 'CMCSA', 'CAG', 'COP', 'ED', 'STZ', 'CEG', 'COO', 'CPRT', 'GLW', 'CPAY', 'CTVA', 'CSGP', 'COST', 'CTRA', 'CRWD', 'CCI', 'CSX', 'CMI', 'CVS', 'DHR', '

## Download the last 10 years of price

In [99]:
# Define the start and end dates for the data
end_date = pd.Timestamp.today()
start_date = end_date - pd.Timedelta(days=365 * 10)

# Download the data
df = yf.download(tickers, start=start_date, end=end_date)

# Print the data (optional)
df.head()

[*********************100%***********************]  503 of 503 completed
ERROR:yfinance:
2 Failed downloads:
ERROR:yfinance:['BF.B']: YFPricesMissingError('$%ticker%: possibly delisted; no price data found  (1d 2014-10-31 03:57:40.528214 -> 2024-10-28 03:57:40.528214)')
ERROR:yfinance:['BRK.B']: YFTzMissingError('$%ticker%: possibly delisted; no timezone found')


Price                      Adj Close                                        \
Ticker                             A       AAPL       ABBV ABNB        ABT   
Date                                                                         
2014-10-31 00:00:00+00:00  36.393883  23.973495  41.956596  NaN  36.026070   
2014-11-03 00:00:00+00:00  37.588360  24.284264  41.817745  NaN  35.778118   
2014-11-04 00:00:00+00:00  36.980892  24.106686  41.427670  NaN  36.042591   
2014-11-05 00:00:00+00:00  36.934872  24.164394  41.348324  NaN  36.083916   
2014-11-06 00:00:00+00:00  38.076145  24.233509  41.460720  NaN  36.108707   

Price                                                                  \
Ticker                          ACGL        ACN       ADBE        ADI   
Date                                                                    
2014-10-31 00:00:00+00:00  18.773333  68.768234  70.120003  40.179718   
2014-11-03 00:00:00+00:00  18.799999  68.641060  69.910004  40.260704   
2014-11-04 00:00:00+00:00  18.910000  68.793655  71.070000  40.042068   
2014-11-05 00:00:00+00:00  18.906668  69.921158  71.370003  40.511715   
2014-11-06 00:00:00+00:00  19.026667  70.565430  72.099998  40.665577   

Price                                 ...  Volume                             \
Ticker                           ADM  ...     WTW       WY     WYNN      XEL   
Date                                  ...                                      
2014-10-31 00:00:00+00:00  35.525322  ...  479916  4841500  1527800  3632900   
2014-11-03 00:00:00+00:00  35.714294  ...  504566  3013800  1222900  3243800   
2014-11-04 00:00:00+00:00  37.445198  ...  440467  2763400  2088000  3709800   
2014-11-05 00:00:00+00:00  38.027206  ...  309965  3056200  2938200  4556000   
2014-11-06 00:00:00+00:00  37.770226  ...  398564  2924200  3081700  8182500   

Price                                                                    \
Ticker                          XOM      XYL      YUM      ZBH     ZBRA   
Date                                                                      
2014-10-31 00:00:00+00:00  17493700  1161000  8379384  1530786   562800   
2014-11-03 00:00:00+00:00  13410000   781100  4277047   813082   833000   
2014-11-04 00:00:00+00:00  13498500   693800  3248402  1050703  1544000   
2014-11-05 00:00:00+00:00  12480000   823700  2750842  1231983  1475600   
2014-11-06 00:00:00+00:00  14731400  1218200  3727741   764054   816600   

Price                                
Ticker                          ZTS  
Date                                 
2014-10-31 00:00:00+00:00   2589500  
2014-11-03 00:00:00+00:00   4071300  
2014-11-04 00:00:00+00:00   9723700  
2014-11-05 00:00:00+00:00  11878900  
2014-11-06 00:00:00+00:00   4457100  

[5 rows x 3018 columns]

## Reshape, and Clean Dataset

In [100]:
# Reshape the DataFrame
df = df['Adj Close'].reset_index()
df = pd.melt(df, id_vars='Date', value_vars=tickers, var_name='Ticker', value_name='Adj Close')
df['Date'] = pd.to_datetime(df['Date']).dt.date
df = df.rename(columns={'index': 'Ticker'})
# Print or use the transformed DataFrame
df.head()

,Date,Ticker,Adj Close
0,2014-10-31,MMM,91.776779
1,2014-11-03,MMM,91.496262
2,2014-11-04,MMM,92.540718
3,2014-11-05,MMM,92.922699
4,2014-11-06,MMM,93.137581


## Calculate Returns

Using log returns instead of simple returns has several advantages, especially in financial analysis:

1. **Time Additivity**
Log returns are time-additive, meaning that you can sum them over different periods to get the total return. For example, if you have daily log returns, you can sum them to find the cumulative return over a week or month.
Simple returns, on the other hand, do not have this property and require a more complex calculation for cumulative returns.
2. **Handling Compounding**
Log returns account for compounding naturally. When returns are compounded, log returns provide a more accurate measure of the growth of an investment over time.
Simple returns can underestimate or overestimate returns if not compounded properly.
3. **Normality Assumption**
Log returns tend to be more normally distributed than simple returns, particularly for large datasets. This property is useful for statistical modeling and risk management.
4. **Symmetry (Time Additivity derived)**
Log returns treat gains and losses symmetrically. A 10% gain followed by a 10% loss results in a net loss in simple returns, but the log returns more accurately reflect the continuous nature of returns.
5. **Ease of Analysis**
Many financial models, like the Black-Scholes option pricing model, rely on the normal distribution, making log returns more compatible with such models.

**Conclusion** <br>
While simple returns are easier to calculate and interpret for short-term analyses, log returns are generally more robust and useful for long-term investments and statistical analyses, especially when compounding and time periods are involved.

In [101]:
# Calculate daily log returns
df['Daily_Log_Return'] = np.log(df['Adj Close'] / df['Adj Close'].shift(1))

# Calculate quarterly log returns
df['Quarterly_Log_Return'] = df.groupby('Ticker')['Daily_Log_Return'].rolling(window=63, min_periods=1).sum().reset_index(0,drop=True)

# Portfolio Optimization: Maximize the Sharpe Ratio

To Maximize a sharpe ratio, you want to maximize the following objective function :
To optimize the Sharpe ratio, you typically want to maximize the following objective function:

### Objective Function
\[ S = \frac{R_p - R_f}{\sigma_p}\]
- **Where**:
  - \(S\) = Sharpe ratio
  - \(R_p\) = Portfolio return
  - \(R_f\) = Risk-free rate
  - \(\sigma_p\) = Portfolio standard deviation (risk)

### Portfolio Return and Risk Formulas
1. **Portfolio Return**:
   \[
   R_p = \sum_{i=1}^{n} w_i \cdot R_i
   \]
   - **Where**:
     - \(w_i\) = Weight of asset \(i\)
     - \(R_i\) = Expected return of asset \(i\)
     - \(n\) = Total number of assets

2. **Portfolio Standard Deviation**:
   \[
   \sigma_p = \sqrt{w^T \Sigma w}
   \]
   - **Where**:
     - \(w\) = Vector of asset weights
     - \(\Sigma\) = Covariance matrix of asset returns

### Optimization Problem
The optimization problem can be framed as follows:

**Maximize**:
\[
S = \frac{\sum_{i=1}^{n} w_i \cdot R_i - R_f}{\sqrt{w^T \Sigma w}}
\]

**Subject to**:
- \(\sum_{i=1}^{n} w_i = 1\) (Weights sum to 1)
- \(w_i \geq 0\) (No short selling, if applicable)

### Implementation
- You would typically use numerical optimization techniques (e.g., gradient ascent or constrained optimization methods) to solve this problem, ensuring that the constraints are respected. Libraries like SciPy in Python can be very helpful for this purpose.

## Calculate Shrink Covariance Matrice

In situations with many variables and skewed data, using a shrinkage covariance matrix is typically the better choice.
Here's why, we will use the Shrinkage Covariance Matrix :

1.  **High Dimensionality:** When the number of variables is large relative to the number of observations, the standard covariance matrix can become unstable and lead to overfitting. Shrinkage helps to stabilize these estimates.

2.   **Skewed Data:** Skewed data can affect the estimation of the covariance matrix. Shrinkage techniques can provide more robust estimates by incorporating prior information or adjusting the influence of extreme values.

3.   **Variance Reduction:** Shrinkage methods reduce the variance of the covariance estimates, which is particularly beneficial in high dimensions. This can improve the reliability of downstream analyses, such as portfolio optimization or classification.

4.  **Bias-Variance Trade-off:** While shrinkage introduces some bias, it often results in a lower mean squared error compared to the traditional covariance matrix, especially in high-dimensional contexts.

**Conclusion** <br>
Given the presence of many variables and skewed data, a shrunk covariance matrix is generally more effective, as it mitigates the instability and variability issues that arise in such scenarios. Techniques like Ledoit-Wolf shrinkage or other regularization methods can be particularly useful.

In [109]:
def calculate_shrink_cov_matrix (df) :
    masked_arr = np.ma.array(df, mask=np.isnan(df))
    cov_numpy = np.ma.cov(masked_arr, rowvar=False, allow_masked=True, ddof=1).data
    n_samples, n_features = df.shape
    alpha = np.mean(cov_numpy**2)
    mu = np.trace(cov_numpy) / n_features
    mu_squared = mu**2
    num = alpha + mu_squared
    den = (n_samples + 1) * (alpha - mu_squared / n_features)
    shrinkage = 1.0 if den == 0 else min(num / den, 1.0)
    shrunk_cov = (1.0 - shrinkage) * cov_numpy
    shrunk_cov.flat[:: n_features + 1] += shrinkage * mu
    return shrunk_cov

returns_matrix = pd.pivot_table(df[['Date','Ticker', 'Quarterly_Log_Return']], index ='Date', columns = 'Ticker', aggfunc='mean')
covariance_matrix = calculate_shrink_cov_matrix(returns_matrix)

## Using Scipy Optimize Solver

In [103]:
def calculate_portfolio_variance(weights, cov_matrix):
    return np.dot(weights.T, np.dot(cov_matrix, weights))

def calculate_portfolio_returns(weights, returns):
    return np.dot(weights, returns)

### Using Negative Sharpe Ratio, as we will use scipy.optimize.minimize
def neg_sharpe_ratio_objective(weights, returns, cov_matrix, risk_free_rate = 0):
    portfolio_returns = np.squeeze(calculate_portfolio_returns(weights, returns))
    portfolio_variance = np.squeeze(calculate_portfolio_variance(weights, cov_matrix))
    return -((portfolio_returns - risk_free_rate)/np.sqrt(portfolio_returns))

In [104]:
# Last Quarter Returns
returns = returns_matrix.iloc[-1].values
#returns_matrix.mean().values.T @ weights # Can you the mean returns (but for 10 years quaterly return, I don't think it makes sense)

# Init guess for the weights
init_guess = np.array([1/len(returns) for _ in range(len(returns))])

result = sp.optimize.minimize(fun=neg_sharpe_ratio_objective,
                            x0=init_guess,
                            args=(returns, covariance_matrix),
                            method='SLSQP',
                            bounds=tuple((0,1) for _ in range(len(returns))),
                            constraints=({'type': 'eq', 'fun': lambda x: np.sum(x) - 1}))

print(result)

 message: Optimization terminated successfully
 success: True
  status: 0
     fun: -0.7462198715083382
       x: [ 0.000e+00  0.000e+00 ...  0.000e+00  4.649e-16]
     nit: 10
     jac: [ 3.495e-02 -4.004e-02 ... -4.821e-02  6.741e-03]
    nfev: 5020
    njev: 10


## Using Convex Solver (CVXPY)

In [139]:
weights = cp.Variable(returns.shape[0])
# gamma = cp.Parameter(nonneg=True)

# Define the objective function (maximize return)
portfolio_return = returns @ weights
portfolio_risk = cp.sum_squares(cp.quad_form(weights, covariance_matrix))
objective = cp.Maximize(portfolio_return)

# Define the constraints
constraints = [
    cp.sum(weights) == 1,
    weights >= 0.005, # not less than 5% of the weight on one asset
    portfolio_risk <= 0.1,
]

# Solve the optimization problem
problem = cp.Problem(objective, constraints)
problem.solve()#qcp=True, verbose= False, solver= 'SCS', eps= 1e-10, max_iters = 100000, warm_start= True)

DCPError: Problem does not follow DCP rules. Specifically:
The following constraints are not DCP:
quad_over_lin(QuadForm(var503804, [[0.04 0.01 ... 0.03 0.03]
 [0.01 0.02 ... 0.01 0.01]
 ...
 [0.03 0.01 ... 0.04 0.02]
 [0.03 0.01 ... 0.02 0.03]]), 1.0) <= 0.1 , because the following subexpressions are not:
|--  QuadForm(var503804, [[0.04 0.01 ... 0.03 0.03]
 [0.01 0.02 ... 0.01 0.01]
 ...
 [0.03 0.01 ... 0.04 0.02]
 [0.03 0.01 ... 0.02 0.03]])